# 1D Scattering Integral
Path calculation has already been done

In [8]:
using Plots; pyplot()
using Formatting
using Cubature

In [9]:
function bars(edges, heights; color=:1, label="", kwargs...)
    n = length(heights)
    
    w = edges[end]-edges[1]
    p = plot(legend=false)
    xlims!(edges[1], edges[end]+0.01*w)
    
    labels = [[label]; repeat([""], inner=[n-1])]
    
    for i in 1:n
        rect = Shape(
            [edges[i],edges[i+1],edges[i+1],edges[i], edges[i]],
            [0.,0.,heights[i],heights[i]]
        )
        
        plot!(rect, color=color, label=labels[i]; kwargs...)
    end
    return p
end

bars (generic function with 1 method)

In [10]:
function integ_grid(s, gₙ, ã)
    N=length(s)
    ds = diff(s)
    result = 0.
    for i = 1:N-1
        bi = -ã[i]*s[i+1] - (i==N-1 ? 0 : sum(ã[j]*ds[j] for j=i+1:N-1))
        if ã[i] == 0
            pcnt_remaining = ds[i]*exp(bi)
        else
            pcnt_remaining = (exp(ã[i]*s[i+1]+bi)-exp(ã[i]*s[i]+bi))/ã[i]
        end
        add = gₙ[i] * pcnt_remaining
        result += gₙ[i] * pcnt_remaining
    end
    
    return result
end

integ_grid (generic function with 1 method)

In [11]:
function integ_num(g, a, lims; abstol=1e-3, reltol=1e-3;)
    f_inner(sp) = g(sp)*exp(-hquadrature(a,sp,lims[2], reltol=reltol, abstol=abstol)[1])
    hquadrature(f_inner, lims..., reltol=reltol, abstol=abstol)[1]
end

integ_num (generic function with 1 method)

# Create random medium

# Calculate explicit ray tracing formula
- Numerical (on grid)
- Compare to symbolic double integral

\begin{equation}
u_n(s) = \int_0^s g(s') \exp\left(-\int_{s'}^{s}a(s'')\, ds''\right)\, ds'
\end{equation}

In [20]:
function test_grid_vs_num(N, g_sym, a_sym)
    # Create random path

    # Random, increasing a starting from 0
    # These are the bin edges
    s = cumsum([[0.]'; rand(N-1)])/N

    # Evaluate quantities on path
    g_num = g_sym.(s)
    a_num = a_sym.(s)
    lims = [minimum(s), maximum(s)];
    
    # Index of s val at or after sp
    î(sp) = (filter(i -> s[i]>sp, 1:N))[1]
    # Distance to next s val
    d̃(sp) = s[î.(sp)]-sp
    
    ds = diff(s)
    Np = 501
    sp = linspace(lims...,Np)[1:end-1]
    dsp = diff(sp)
    gₙ = g_sym.(sp)
    ã = a_sym.(sp)
    #minus_ν = [sum((a_num.*ds)[î(sp[i]):end]) + a_num[î(sp[i])-1]*d̃(sp[i]) for i in 1:length(sp)]
    #minus_ν = [sum((ã.*dsp)[î(sp[i]):end]) + ã[î(sp[i])-1]*d̃(sp[i]) for i in 1:Np]
    #
    ## Plot quantities
    #bars(s, ã, label="absorb")
    #plot!(sp,minus_ν, color=:green, marker=:circle, label="atten")
    #plot!(sp,exp.(-minus_ν), color=:red, marker=:circle, label="% remaining")
    #plot!(legend=true)
    #xlabel!("Path length (ray heading right)")
    #title!("Inner path integral")
    
    # Compare grid to numerical solution
    i_g = integ_grid(s, g_num, a_num)
    i_n = integ_num(g_sym, a_sym, lims)
    abs_err = abs(i_n - i_g)
    rel_err = abs_err/i_g

    println("i_g = $(i_g)")
    println("i_n = $(i_n)")
    printfmtln("Absolute error = {:.2f}", abs_err)
    printfmtln("Relative error = {:.2f}%", 100*rel_err)
end


test_grid_vs_num (generic function with 1 method)

In [32]:
N = 20
g_sym(s) = abs(sin(s)*cos(s))
a_sym(s) = abs(s^2-s)+.3
test_grid_vs_num(N, g_sym, a_sym)

i_g = 0.07245580126209929
i_n = 0.0781990153801958
Absolute error = 0.01
Relative error = 7.93%
